# Portfolio Analyzer

In [8]:
pip install polygon-api-client

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#from polygon import RESTClient

In [9]:
#import the necessary modules
import pandas as pd
#questionary
from pathlib import Path
import fire
import questionary
#API
import os
import json
import requests
from dotenv import load_dotenv
from MCForecastTools import MCSimulation
%matplotlib inline


In [5]:
from IPython.display import display_html 

In [12]:
POLYGON_API_KEY ='enZp2AUpH4pGXJJuQ1CbjdVXJIsBFBEl'

env: POLYGON_API_KEY='enZp2AUpH4pGXJJuQ1CbjdVXJIsBFBEl'


In [10]:
# Load .env enviroment variables into the notebook
#load_dotenv()

False

In [11]:
# Get the API key from the environment variable and store as Python variable
#polygon_api_key = os.getenv("POLYGON_API_KEY")

# Using the type funcion, confirm that the Nasdaq API key is available for use in the notebook
#type(polygon_api_key)

In [ ]:
ticker_list = ["AMZN", "RTH","AMT", "IYR", "XOM", "XLE"]

In [78]:
import requests
import datetime
from time import sleep
from tqdm import tqdm
def get_prices(start_date,end_date,universe):
    data = pd.DataFrame()
    # end_at = datetime.datetime.now() 
    # begin_from = end_at + datetime.timedelta(days=-5000)
    for t in tqdm(universe):
        print(t)
        print(len(data))
        # for start in pd.date_range(start = begin_from, end = end_at,normalize=True,freq = '88D'):
        #     end = start + datetime.timedelta(days = 88)
        #     end = str(int((end - datetime.datetime(1970,1,1)).total_seconds()))
        #     start = str(int((start - datetime.datetime(1970,1,1)).total_seconds()))
        try:
            r = requests.get(f'https://api.polygon.io/v2/aggs/ticker/{t}/range/1/day/{start_date}/{end_date}?apiKey={POLYGON_API_KEY}')
        except json.JSONDecodeError:
            pass
        try:
            # print(r.json())
            temp_data = pd.DataFrame(r.json()['results'])
            temp_data['ticker'] = t
            data = data.append(temp_data, ignore_index=True)
            print(t+':'+str(pd.to_datetime(start_date, unit='s'))+':'+str(pd.to_datetime(end_date, unit='s')))
        except KeyError:
            print(f'{t} was not found')
            pass
        except ValueError:
            pass
        except ConnectionError as error:
            print(error)
            sleep(200)
            continue
        except TimeoutError as error:
            print(error)
            sleep(200)
            continue
        sleep(0.25)
#     data.to_csv(file_name,index = False)
    data.t = pd.to_datetime(data.t, unit = 'ms')
    columns_name = {'t': 'time', 'o': 'Open', 'c': 'Close', 'h': 'High', 'l': 'Low', 'v': 'Volume'} 
    data= data.rename(columns = columns_name)
    data = data[['time','ticker','Open', 'Close','High', 'Low', 'Volume']]
    data = data.set_index(['time', 'ticker'])
    data = data.pct_change()
    data = data.dropna()
    #data.to_csv(file_name, index = False)
    return data

In [79]:
#f"https://api.polygon.io/v2/aggs/ticker/{t}/range/1/day/{start_date}/{end_date}?apiKey=rHtIrGbGhJY6umnXSqp6hYgTM3XnHI2_"

In [105]:
# Create `start_date` and `end_date` variables for the period before the pandemic
start_date = "2017-03-01"
end_date = "2020-03-01"
all_data = get_prices(start_date=start_date, end_date=end_date, universe=ticker_list)
#example
#big_mac_usa = "https://data.nasdaq.com/api/v3/datasets/ECONOMIST/BIGMAC_USA?start_date=" + start_date + "&end_date=" + end_date + "&api_key=" + nasdaq_api_key

  0%|                                                     | 0/6 [00:00<?, ?it/s]

AMZN
0


 17%|███████▌                                     | 1/6 [00:00<00:04,  1.11it/s]

RTH
575


 33%|███████████████                              | 2/6 [00:01<00:03,  1.29it/s]

...

 67%|██████████████████████████████               | 4/6 [00:02<00:01,  1.42it/s]

XOM
2300


 83%|█████████████████████████████████████▌       | 5/6 [00:03<00:00,  1.45it/s]

XLE
2875


100%|█████████████████████████████████████████████| 6/6 [00:04<00:00,  1.41it/s]


In [106]:
all_data.index

MultiIndex([('2017-11-15 05:00:00', 'AMZN'),
            ('2017-11-16 05:00:00', 'AMZN'),
            ('2017-11-17 05:00:00', 'AMZN'),
            ('2017-11-20 05:00:00', 'AMZN'),
            ('2017-11-21 05:00:00', 'AMZN'),
            ('2017-11-22 05:00:00', 'AMZN'),
            ('2017-11-24 05:00:00', 'AMZN'),
            ('2017-11-27 05:00:00', 'AMZN'),
            ('2017-11-28 05:00:00', 'AMZN'),
            ('2017-11-29 05:00:00', 'AMZN'),
            ...
            ('2020-02-14 05:00:00',  'XLE'),
            ('2020-02-18 05:00:00',  'XLE'),
            ('2020-02-19 05:00:00',  'XLE'),
            ('2020-02-20 05:00:00',  'XLE'),
            ('2020-02-21 05:00:00',  'XLE'),
            ('2020-02-24 05:00:00',  'XLE'),
            ('2020-02-25 05:00:00',  'XLE'),
            ('2020-02-26 05:00:00',  'XLE'),
            ('2020-02-27 05:00:00',  'XLE'),
            ('2020-02-28 05:00:00',  'XLE')],
           names=['time', 'ticker'], length=3449)

In [107]:
idx = pd.IndexSlice

In [108]:
AMZN_daily_returns_df = all_data.loc[idx[:,'AMZN'],['Close']]
AMZN_daily_returns_df

,,Close
time,ticker,
2017-11-15 05:00:00,AMZN,-0.008928
2017-11-16 05:00:00,AMZN,0.009408
2017-11-17 05:00:00,AMZN,-0.006515
2017-11-20 05:00:00,AMZN,-0.003160
2017-11-21 05:00:00,AMZN,0.011702
...,...,...
2020-02-24 05:00:00,AMZN,-0.041356
2020-02-25 05:00:00,AMZN,-0.018191
2020-02-26 05:00:00,AMZN,0.003472


In [109]:
RTH_daily_returns_df = all_data.loc[idx[:,'RTH'],['Close']]
RTH_daily_returns_df

,,Close
time,ticker,
2017-11-14 05:00:00,RTH,-0.114107
2017-11-15 05:00:00,RTH,-0.009228
2017-11-16 05:00:00,RTH,0.020080
2017-11-17 05:00:00,RTH,0.009961
2017-11-20 05:00:00,RTH,-0.000587
...,...,...
2020-02-24 05:00:00,RTH,-0.026701
2020-02-25 05:00:00,RTH,-0.023793
2020-02-26 05:00:00,RTH,-0.001961


In [110]:
AMT_daily_returns_df = all_data.loc[idx[:,'AMT'],['Close']]
AMT_daily_returns_df

,,Close
time,ticker,
2017-11-14 05:00:00,AMT,0.328564
2017-11-15 05:00:00,AMT,-0.003746
2017-11-16 05:00:00,AMT,-0.003559
2017-11-17 05:00:00,AMT,-0.017049
2017-11-20 05:00:00,AMT,0.002399
...,...,...
2020-02-24 05:00:00,AMT,-0.007692
2020-02-25 05:00:00,AMT,0.004202
2020-02-26 05:00:00,AMT,-0.014098


In [111]:
IYR_daily_returns_df = all_data.loc[idx[:,'IYR'],['Close']]
IYR_daily_returns_df

,,Close
time,ticker,
2017-11-14 05:00:00,IYR,-0.635450
2017-11-15 05:00:00,IYR,-0.008587
2017-11-16 05:00:00,IYR,0.006710
2017-11-17 05:00:00,IYR,-0.004605
2017-11-20 05:00:00,IYR,-0.002070
...,...,...
2020-02-24 05:00:00,IYR,-0.013372
2020-02-25 05:00:00,IYR,-0.026803
2020-02-26 05:00:00,IYR,-0.010497


In [112]:
XOM_daily_returns_df = all_data.loc[idx[:,'XOM'],['Close']]
XOM_daily_returns_df

,,Close
time,ticker,
2017-11-14 05:00:00,XOM,-0.059039
2017-11-15 05:00:00,XOM,-0.012524
2017-11-16 05:00:00,XOM,-0.008004
2017-11-17 05:00:00,XOM,-0.003972
2017-11-20 05:00:00,XOM,0.003863
...,...,...
2020-02-24 05:00:00,XOM,-0.046846
2020-02-25 05:00:00,XOM,-0.038325
2020-02-26 05:00:00,XOM,-0.021956


In [113]:
XLE_daily_returns_df = all_data.loc[idx[:,'XLE'],['Close']]
XLE_daily_returns_df

,,Close
time,ticker,
2017-11-14 05:00:00,XLE,0.324650
2017-11-15 05:00:00,XLE,-0.011300
2017-11-16 05:00:00,XLE,-0.004008
2017-11-17 05:00:00,XLE,0.004918
2017-11-20 05:00:00,XLE,-0.002373
...,...,...
2020-02-24 05:00:00,XLE,-0.046177
2020-02-25 05:00:00,XLE,-0.044152
2020-02-26 05:00:00,XLE,-0.030186


In [114]:
tech_daily_returns = pd.concat([AMZN_daily_return_df, RTH_daily_return_df], keys=["AMZN","RTH"])
tech_daily_returns

Close
     time                ticker          
AMZN 2017-11-15 05:00:00 AMZN   -0.008928
     2017-11-16 05:00:00 AMZN    0.009408
     2017-11-17 05:00:00 AMZN   -0.006515
     2017-11-20 05:00:00 AMZN   -0.003160
     2017-11-21 05:00:00 AMZN    0.011702
...                                   ...
RTH  2020-02-24 05:00:00 RTH    -0.026701
     2020-02-25 05:00:00 RTH    -0.023793
     2020-02-26 05:00:00 RTH    -0.001961
     2020-02-27 05:00:00 RTH    -0.041607
     2020-02-28 05:00:00 RTH    -0.017861

[1148 rows x 1 columns]

In [115]:
RE_daily_returns = pd.concat([AMT_daily_returns_df, IYR_daily_returns_df], keys=["AMT","IYR"])
RE_daily_returns

Close
    time                ticker          
AMT 2017-11-14 05:00:00 AMT     0.328564
    2017-11-15 05:00:00 AMT    -0.003746
    2017-11-16 05:00:00 AMT    -0.003559
    2017-11-17 05:00:00 AMT    -0.017049
    2017-11-20 05:00:00 AMT     0.002399
...                                  ...
IYR 2020-02-24 05:00:00 IYR    -0.013372
    2020-02-25 05:00:00 IYR    -0.026803
    2020-02-26 05:00:00 IYR    -0.010497
    2020-02-27 05:00:00 IYR    -0.052410
    2020-02-28 05:00:00 IYR    -0.031257

[1150 rows x 1 columns]

In [116]:
energy_daily_returns = pd.concat([XOM_daily_returns_df, XLE_daily_returns_df], keys=["XOM","XLE"])
energy_daily_returns

Close
    time                ticker          
XOM 2017-11-14 05:00:00 XOM    -0.059039
    2017-11-15 05:00:00 XOM    -0.012524
    2017-11-16 05:00:00 XOM    -0.008004
    2017-11-17 05:00:00 XOM    -0.003972
    2017-11-20 05:00:00 XOM     0.003863
...                                  ...
XLE 2020-02-24 05:00:00 XLE    -0.046177
    2020-02-25 05:00:00 XLE    -0.044152
    2020-02-26 05:00:00 XLE    -0.030186
    2020-02-27 05:00:00 XLE    -0.053478
    2020-02-28 05:00:00 XLE    -0.000883

[1150 rows x 1 columns]

In [117]:
#Display all sectors of the stock/ETF dataframes
dfT_style = tech_daily_returns.style.set_table_attributes("style='display:inline; margin-right:20px;'").set_caption("Tech")
dfR_style = RE_daily_returns.style.set_table_attributes("style='display:inline'").set_caption("Real Estate")
dfE_style = energy_daily_returns.style.set_table_attributes("style='display:inline'").set_caption("Energy")


display_html(dfT_style._repr_html_() + dfR_style._repr_html_() + dfE_style._repr_html_(), raw=True)

Tech 
 
 
   
   
   
 Close 
 
 
   
 time 
 ticker 
   
 
 
 
 
 AMZN 
 2017-11-15 05:00:00 
 AMZN 
 -0.008928 
 
 
 2017-11-16 05:00:00 
 AMZN 
 0.009408 
 
 
 2017-11-17 05:00:00 
 AMZN 
 -0.006515 
 
 
 2017-11-20 05:00:00 
 AMZN 
 -0.003160 
 
 
 2017-11-21 05:00:00 
 AMZN 
 0.011702 
 
 
 2017-11-22 05:00:00 
 AMZN 
 0.014629 
 
 
 2017-11-24 05:00:00 
 AMZN 
 0.025810 
 
 
 2017-11-27 05:00:00 
 AMZN 
 0.008288 
 
 
 2017-11-28 05:00:00 
 AMZN 
 -0.001865 
 
 
 2017-11-29 05:00:00 
 AMZN 
 -0.027086 
 
 
 2017-11-30 05:00:00 
 AMZN 
 0.013330 
 
 
 2017-12-01 05:00:00 
 AMZN 
 -0.012237 
 
 
 2017-12-04 05:00:00 
 AMZN 
 -0.024433 
 
 
 2017-12-05 05:00:00 
 AMZN 
 0.006720 
 
 
 2017-12-06 05:00:00 
 AMZN 
 0.009443 
 
 
 2017-12-07 05:00:00 
 AMZN 
 0.006456 
 
 
 2017-12-08 05:00:00 
 AMZN 
 0.001906 
 
 
 2017-12-11 05:00:00 
 AMZN 
 0.005955 
 
 
 2017-12-12 05:00:00 
 AMZN 
 -0.003285 
 
 
 2017-12-13 05:00:00 
 AMZN 
 -0.000815 
 
 
 2017-12-14 05:00:00 
 AMZN 
 0.008702 
 
 
 2017-12-15 05:00:00 
 AMZN 
 0.004156 
 
 
 2017-12-18 05:00:00 
 AMZN 
 0.009702 
 
 
 2017-12-19 05:00:00 
 AMZN 
 -0.002688 
 
 
 2017-12-20 05:00:00 
 AMZN 
 -0.008220 
 
 
 2017-12-21 05:00:00 
 AMZN 
 -0.002429 
 
 
 2017-12-22 05:00:00 
 AMZN 
 -0.005448 
 
 
 2017-12-26 05:00:00 
 AMZN 
 0.007190 
 
 
 2017-12-27 05:00:00 
 AMZN 
 0.004674 
 
 
 2017-12-28 05:00:00 
 AMZN 
 0.003248 
 
 
 2017-12-29 05:00:00 
 AMZN 
 -0.014021 
 
 
 2018-01-02 05:00:00 
 AMZN 
 0.016708 
 
 
 2018-01-03 05:00:00 
 AMZN 
 0.012775 
 
 
 2018-01-04 05:00:00 
 AMZN 
 0.004476 
 
 
 2018-01-05 05:00:00 
 AMZN 
 0.016163 
 
 
 2018-01-08 05:00:00 
 AMZN 
 0.014425 
 
 
 2018-01-09 05:00:00 
 AMZN 
 0.004676 
 
 
 2018-01-10 05:00:00 
 AMZN 
 0.001301 
 
 
 2018-01-11 05:00:00 
 AMZN 
 0.017818 
 
 
 2018-01-12 05:00:00 
 AMZN 
 0.022339 
 
 
 2018-01-16 05:00:00 
 AMZN 
 -0.000260 
 
 
 2018-01-17 05:00:00 
 AMZN 
 -0.007556 
 
 
 2018-01-18 05:00:00 
 AMZN 
 -0.001297 
 
 
 2018-01-19 05:00:00 
 AMZN 
 0.000974 
 
 
 2018-01-22 05:00:00 
 AMZN 
 0.025282 
 
 
 2018-01-23 05:00:00 
 AMZN 
 0.026542 
 
 
 2018-01-24 05:00:00 
 AMZN 
 -0.003692 
 
 
 2018-01-25 05:00:00 
 AMZN 
 0.015057 
 
 
 2018-01-26 05:00:00 
 AMZN 
 0.017490 
 
 
 2018-01-29 05:00:00 
 AMZN 
 0.011148 
 
 
 2018-01-30 05:00:00 
 AMZN 
 0.014206 
 
 
 2018-01-31 05:00:00 
 AMZN 
 0.009090 
 
 
 2018-02-01 05:00:00 
 AMZN 
 -0.041967 
 
 
 2018-02-02 05:00:00 
 AMZN 
 0.028741 
 
 
 2018-02-05 05:00:00 
 AMZN 
 -0.027938 
 
 
 2018-02-06 05:00:00 
 AMZN 
 0.038014 
 
 
 2018-02-07 05:00:00 
 AMZN 
 -0.018062 
 
 
 2018-02-08 05:00:00 
 AMZN 
 -0.046782 
 
 
 2018-02-09 05:00:00 
 AMZN 
 -0.008071 
 
 
 2018-02-12 05:00:00 
 AMZN 
 0.034809 
 
 
 2018-02-13 05:00:00 
 AMZN 
 0.020401 
 
 
 2018-02-14 05:00:00 
 AMZN 
 0.025832 
 
 
 2018-02-15 05:00:00 
 AMZN 
 0.007381 
 
 
 2018-02-16 05:00:00 
 AMZN 
 -0.008941 
 
 
 2018-02-20 05:00:00 
 AMZN 
 0.013571 
 
 
 2018-02-21 05:00:00 
 AMZN 
 0.009923 
 
 
 2018-02-22 05:00:00 
 AMZN 
 0.001632 
 
 
 2018-02-23 05:00:00 
 AMZN 
 0.009870 
 
 
 2018-02-26 05:00:00 
 AMZN 
 0.014633 
 
 
 2018-02-27 05:00:00 
 AMZN 
 -0.006551 
 
 
 2018-02-28 05:00:00 
 AMZN 
 0.000311 
 
 
 2018-03-01 05:00:00 
 AMZN 
 -0.012562 
 
 
 2018-03-02 05:00:00 
 AMZN 
 0.004553 
 
 
 2018-03-05 05:00:00 
 AMZN 
 0.015571 
 
 
 2018-03-06 05:00:00 
 AMZN 
 0.009208 
 
 
 2018-03-07 05:00:00 
 AMZN 
 0.004787 
 
 
 2018-03-08 05:00:00 
 AMZN 
 0.004440 
 
 
 2018-03-09 05:00:00 
 AMZN 
 0.017418 
 
 
 2018-03-12 04:00:00 
 AMZN 
 0.012350 
 
 
 2018-03-13 04:00:00 
 AMZN 
 -0.006388 
 
 
 2018-03-14 04:00:00 
 AMZN 
 0.001776 
 
 
 2018-03-15 04:00:00 
 AMZN 
 -0.005456 
 
 
 2018-03-16 04:00:00 
 AMZN 
 -0.006724 
 
 
 2018-03-19 04:00:00 
 AMZN 
 -0.017020 
 
 
 2018-03-20 04:00:00 
 AMZN 
 0.026914 
 
 
 2018-03-21 04:00:00 
 AMZN 
 -0.002931 
 
 
 2018-03-22 04:00:00 
 AMZN 
 -0.023352 
 
 
 2018-03-23 04:00:00 
 AMZN 
 -0.031950 
 
 
 2018-03-26 04:00:00 


### A) Analyzing Each Stock and ETF Individually


In [ ]:
#Summary statistics for AMZN stock using .std() and .mean()


In [ ]:
#Summary statistics for RTH stock using .std() and .mean()



In [ ]:
#Summary statistics for AMT stock using .std() and .mean()



In [ ]:
#Summary statistics for IYR stock using .std() and .mean()



In [ ]:
#Summary statistics for XOM stock using .std() and .mean()



In [ ]:
#Summary statistics for XLE stock using .std() and .mean()



In [ ]:
#Summary statistics for each stock using .describe()



In [ ]:
# Variance For Each Stock and ETF


In [ ]:
# Covariance for Each Stock and ETF

In [ ]:
#Sharpe ratio for each stock and ETF 


In [ ]:
# Calculate betas of AMZN, RTH, AMT, IYR, XOM, XLE 

# Display the beta of all stocks and ETFS


In [ ]:
# Calculate alphas of AMZN, RTH, AMT, IYR, XOM, XLE 

# Display the alphas of all stocks and ETFS


### A) Analyzing What Would Happen To A Portfolio in Each Sector For the Period Before the Pandemic

In [ ]:
#Questionary SQL
#what kind of investor are you (risk-averse, risk-neutral, risk loving)?
#assign portfolio to appropriate risk level given: risk-averse-all indexes, risk-neutral:mix, risk loving:all stocks

#What percentage of Tech would you like in your portfolio?
#What percentage of Real Estate would you like in your portfolio?
#What percentage of Energy would you like in your portfolio?

#What is your time horizon for investment? (short is more safe, long is riskier?)


In [ ]:
#Portfolio Returns for Tech: use (portfolio_return = weightAMZN * meanAMZN + weightRTH * meanRTH)


In [ ]:
#Portfolio Returns for Real Estate: use (portfolio_return = weightAMT * meanAMT + weightIYR * meanIYR)



In [ ]:
#Portfolio Returns for Energy: use (portfolio_return = weightXOM * meanXOM + weightXLE * meanXLE)



In [ ]:
# Using the Pandas var function, calculate the covariance of the S&P 500 using tech portfolio returns information
# The ETF SPY will represent the market


In [ ]:
# Using the Pandas var function, calculate the covariance of the S&P 500 using Real Estate portfolio returns information
# The ETF SPY will represent the market


In [ ]:
# Using the Pandas var function, calculate the covariance of the S&P 500 using Energy portfolio returns information
# The ETF SPY will represent the market


In [ ]:
#Calculate the Portfolio Standard Deviation for Tech: use (weightAMZN^2 * MeanAMZN^2 + weightRTH^2 * MeanRTH^2 + 2*weightAMZN*weightRTH*stdevAMZN*stdevRTH*covAMZN,RTH)


In [ ]:
#Calculate the Portfolio Standard Deviation for Real Estate


In [ ]:
#Calculate the Portfolio Standard Deviation for Energy


In [ ]:
# Using the Pandas rolling function in conjunction with the var function, 
# calculate the 30-day rolling variance for the S&P 500 using tech daily returns information

# Visualize the 30-day rolling variance of the S&P 500
# Be sure to adjust the figure size and add a title


In [ ]:
#Monte Carlo Simulation


In [ ]:
#Box and Whisker Plot


In [ ]:
#HVPlot


In [ ]:
#bar plot comparing returns of all sectors 

In [104]:
#bar plot comparing returns of all portfolio